In [1]:
from warnings import filterwarnings

filterwarnings('ignore')

In [2]:
from copy import deepcopy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from more_itertools import flatten

In [3]:
import networkx as nx

In [4]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [5]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [37]:
# net_df denotes a dataset that has had values prior to 1946 removed
alliance_net_df = pd.read_pickle('pickle/alliance_net_df.pkl')
mid_net_df = pd.read_pickle('pickle/mid_net_df.pkl')
trade_net_df = pd.read_pickle('pickle/trade_net_df.pkl')
trade_df = pd.read_pickle('pickle/trade_df.pkl')
gov_df = pd.read_pickle('pickle/gov_df.pkl')
cait_df = pd.read_pickle('pickle/cait_df.pkl')

In [38]:
aggregations = {
    'year': 'count'
    }

country_loc_df = gov_df.groupby(['c_code_1', 'un_region', 'un_continent']).agg(aggregations).reset_index()
country_loc_df.rename({'year': 'num_years'}, axis = 1, inplace = True)

In [39]:
country_loc_df.loc[197, 'c_code_1'] = 817
country_loc_df.loc[197, 'un_region'] = 'South-Eastern Asia'
country_loc_df.loc[197, 'un_continent'] = 'Asia'
country_loc_df.loc[197, 'num_years'] = 0
country_loc_df.loc[198, 'c_code_1'] = 347
country_loc_df.loc[198, 'un_region'] = 'Southern Europe'
country_loc_df.loc[198, 'un_continent'] = 'Europe'
country_loc_df.loc[198, 'num_years'] = 0
country_loc_df.loc[199, 'c_code_1'] = 511
country_loc_df.loc[199, 'un_region'] = 'Eastern Africa'
country_loc_df.loc[199, 'un_continent'] = 'Africa'
country_loc_df.loc[199, 'num_years'] = 0
country_loc_df.loc[200, 'c_code_1'] = 221
country_loc_df.loc[200, 'un_region'] = 'Western Europe'
country_loc_df.loc[200, 'un_continent'] = 'Europe'
country_loc_df.loc[200, 'num_years'] = 0

In [40]:
trade_net_df = deepcopy(pd.merge(country_loc_df, trade_net_df, how = 'inner', on = ['c_code_1']))
trade_net_df.rename({'un_region': 'un_region_1', 'un_continent': 'un_continent_1'}, axis = 1, inplace = True)
trade_net_df.drop(['num_years'], axis = 1, inplace = True)

In [41]:
country_loc_df.rename({'c_code_1': 'c_code_2'}, axis = 1, inplace = True)

In [42]:
trade_net_df = deepcopy(pd.merge(country_loc_df, trade_net_df, how = 'inner', on = ['c_code_2']))
trade_net_df.rename({'un_region': 'un_region_2', 'un_continent': 'un_continent_2'}, axis = 1, inplace = True)
trade_net_df.drop(['num_years'], axis = 1, inplace = True)

In [43]:
print(len(trade_net_df[trade_net_df['money_flow_1'] <= 0]))
print(len(trade_net_df[trade_net_df['money_flow_2'] <= 0]))

0
0


In [44]:
trade_net_df.head()

,c_code_2,un_region_2,un_continent_2,c_code_1,un_region_1,un_continent_1,money_flow_1,money_flow_2,state_name_1,state_name_2,year
0,2.000,Northern America,Americas,20.000,Northern America,Americas,2038500000.000,1794000000.000,Canada,United States of America,1946
1,2.000,Northern America,Americas,20.000,Northern America,Americas,2038500000.000,1794000000.000,Canada,United States of America,1947
2,2.000,Northern America,Americas,20.000,Northern America,Americas,2038500000.000,1794000000.000,Canada,United States of America,1948
3,2.000,Northern America,Americas,20.000,Northern America,Americas,2101200000.000,1743000000.000,Canada,United States of America,1949
4,2.000,Northern America,Americas,20.000,Northern America,Americas,2162100000.000,2101000000.000,Canada,United States of America,1950


In [45]:
alliance_net_df.head()

,c_code_1,state_name_1,c_code_2,state_name_2,pre_1816_alliance,in_effect_1231_2012,defense_treaty,neutrality_treaty,nonaggression_treaty,entente_treaty,year,alliance_date_start,alliance_date_end
14772,91,Honduras,150,Paraguay,0,0,1,0,0.000,1.000,1946,1945-03-06,1947-09-02
14773,651,Egypt,200,United Kingdom,0,0,0,0,0.000,1.000,1946,1937-05-26,1951-10-08
14774,91,Honduras,94,Costa Rica,0,0,1,0,0.000,1.000,1946,1945-03-06,1947-09-02
14775,93,Nicaragua,145,Bolivia,0,0,1,0,0.000,1.000,1946,1945-03-06,1947-09-02
14776,365,Russia,712,Mongolia,0,0,1,0,0.000,0.000,1946,1936-03-12,1991-12-25


In [46]:
mid_net_df.head()

,dispute_id,c_code_1,state_name_abb_a,c_code_2,state_name_abb_b,year,outcome,settlement_type,total_fatality_bin,num_with,num_against,revision_sought,revision_type,state_fatality_bin,hostility_level,origin_participant,war,year_num,cumulative_duration,ongoing_2010,date_start,date_end,initiator
3544,1792,220,FRN,800,THI,1946,Stalemate,Negotiated,101-250,1,1,0,N/A,1-25,4,1,0,1,251,0,1946-04-01,1946-12-07,1.000
3545,1792,800,THI,220,FRN,1946,Stalemate,Negotiated,101-250,1,1,0,N/A,101-250,4,1,0,1,251,0,1946-04-01,1946-12-07,0.000
3546,2704,710,CHN,365,RUS,1946,Unclear,Unclear,None Value,1,1,0,N/A,None Value,1,1,0,2,211,0,1945-10-06,1946-05-03,0.000
3547,3192,200,UKG,345,YUG,1946,Stalemate,None Value,None Value,1,1,0,N/A,None Value,4,1,0,1,1,0,1946-10-05,1946-10-05,1.000
3548,3192,345,YUG,200,UKG,1946,Stalemate,None Value,None Value,1,1,0,N/A,None Value,4,1,0,1,1,0,1946-10-05,1946-10-05,0.000


Top-Level Network Statistics with Pandas

In [47]:
# instances of trade, alliances, and conflicts across all years.
# divide by two because all rows are duplicated so each country can be c_code_1 and c_code2
print('trade:', int(len(trade_net_df)/2))
print('alliances:', int(len(alliance_net_df)/2))
print('conflict:', int(len(mid_net_df)/2))

trade: 703116
alliances: 66743
conflict: 3154


In [48]:
mid_net_df[mid_net_df['date_end'].astype(str) == 'Nat']

,dispute_id,c_code_1,state_name_abb_a,c_code_2,state_name_abb_b,year,outcome,settlement_type,total_fatality_bin,num_with,num_against,revision_sought,revision_type,state_fatality_bin,hostility_level,origin_participant,war,year_num,cumulative_duration,ongoing_2010,date_start,date_end,initiator


In [49]:
aggregations = {'year': 'count'}
trade_net_df_grouped = trade_net_df.groupby(['c_code_1', 'c_code_2']).agg(aggregations).reset_index()
trade_net_df_grouped.drop(['year'], axis = 1, inplace = True)
alliance_net_df_grouped = alliance_net_df.groupby(['c_code_1', 'c_code_2', 'alliance_date_start']).agg(aggregations).reset_index()
alliance_net_df_grouped.drop(['year'], axis = 1, inplace = True)
mid_net_df_grouped = mid_net_df.groupby(['c_code_1', 'c_code_2', 'dispute_id', 'date_start', 'date_end']).agg(aggregations).reset_index()
mid_net_df_grouped.drop(['year'], axis = 1, inplace = True)
# total distinct trading parties, alliances, conflicts (between two countries) in dataset.
# divide by two because all rows are duplicated so each country can be c_code_1 and c_code2
print('distinct trading country combinations:', int(len(trade_net_df_grouped)/2))
print('distinct alliances:', int(len(alliance_net_df_grouped)/2))
print('distinct conflicts:', int(len(mid_net_df_grouped)/2))

distinct trading country combinations: 19815
distinct alliances: 2483
distinct conflicts: 2158


In [50]:
# total countries forming alliances in dataset.
assert len(trade_net_df['c_code_1'].unique()) == len(trade_net_df_grouped['c_code_1'].unique())
assert len(alliance_net_df['c_code_1'].unique()) == len(alliance_net_df_grouped['c_code_1'].unique())
assert len(mid_net_df['c_code_1'].unique()) == len(mid_net_df_grouped['c_code_1'].unique())
print('total countries engaging in trade:', int(len(trade_net_df['c_code_1'].unique())))
print('total countries forming alliances:', int(len(alliance_net_df['c_code_1'].unique())))
print('total countries involved in conflict:', int(len(mid_net_df['c_code_1'].unique())))

total countries engaging in trade: 201
total countries forming alliances: 164
total countries involved in conflict: 176


In [51]:
trade_ally = pd.merge(trade_net_df, alliance_net_df, how = 'inner', on = ['c_code_1', 'c_code_2', 'year'])
ally_mid = pd.merge(alliance_net_df, mid_net_df, how = 'inner', on = ['c_code_1', 'c_code_2', 'year'])
trade_mid = pd.merge(trade_net_df, mid_net_df, how = 'inner', on = ['c_code_1', 'c_code_2', 'year'])
trade_ally_mid = pd.merge(trade_ally, mid_net_df, how = 'inner', on = ['c_code_1', 'c_code_2', 'year'])
# instances of simultaneous trade and alliance, simultaneous alliance and conflict,
# simultaneous trade and conflict, simultaneous trade, alliance an conflict.
# divide by two because all rows are duplicated so each country can be c_code_1 and c_code2
print('instance of simultaneous trade and alliance:', int(len(trade_ally)/2))
print('instance of simultaneous alliance and conflict:', int(len(ally_mid)/2))
print('instance of simultaneous trade and conflict:', int(len(trade_mid)/2))
print('instance of simultaneous trade, alliance, and conflict:', int(len(trade_ally_mid)/2))

instance of simultaneous trade and alliance: 61567
instance of simultaneous alliance and conflict: 1105
instance of simultaneous trade and conflict: 3125
instance of simultaneous trade, alliance, and conflict: 1083


In [52]:
ally_mid = deepcopy(pd.merge(country_loc_df, ally_mid, how = 'inner', on = ['c_code_2']))
ally_mid.rename({'un_region': 'un_region_2', 'un_continent': 'un_continent_2'}, axis = 1, inplace = True)
ally_mid.drop(['num_years'], axis = 1, inplace = True)

In [53]:
country_loc_df.rename({'c_code_2': 'c_code_1'}, axis = 1, inplace = True)

In [54]:
ally_mid = deepcopy(pd.merge(country_loc_df, ally_mid, how = 'inner', on = ['c_code_1']))
ally_mid.rename({'un_region': 'un_region_1', 'un_continent': 'un_continent_1'}, axis = 1, inplace = True)
ally_mid.drop(['num_years'], axis = 1, inplace = True)

In [55]:
total_trades = int(len(trade_net_df)/2)
total_trading_co = int(len(trade_net_df['c_code_1'].unique()))
total_alliances = int(len(alliance_net_df)/2)
total_alliance_co = int(len(alliance_net_df['c_code_1'].unique()))
total_conflicts = int(len(mid_net_df)/2)
total_conflict_co = int(len(mid_net_df['c_code_1'].unique()))

In [56]:
simul_trade_ally = int(len(trade_ally)/2)
simul_trade_ally_reg = int(len(trade_ally[trade_ally['un_region_1'] == trade_ally['un_region_2']])/2)
simul_trade_ally_cont = int(len(trade_ally[trade_ally['un_continent_1'] == trade_ally['un_continent_2']])/2)
simul_trade_ally_co = int(len(trade_ally['c_code_1'].unique()))
simul_trade_mid = int(len(trade_mid)/2)
simul_trade_mid_reg = int(len(trade_mid[trade_mid['un_region_1'] == trade_mid['un_region_2']])/2)
simul_trade_mid_cont = int(len(trade_mid[trade_mid['un_continent_1'] == trade_mid['un_continent_2']])/2)
simul_trade_mid_co = int(len(trade_mid['c_code_1'].unique()))
simul_ally_mid = int(len(ally_mid)/2)
simul_ally_mid_reg = int(len(ally_mid[ally_mid['un_region_1'] == ally_mid['un_region_2']])/2)
simul_ally_mid_cont = int(len(ally_mid[ally_mid['un_continent_1'] == ally_mid['un_continent_2']])/2)
simul_ally_mid_co = int(len(ally_mid['c_code_1'].unique()))
simul_trade_ally_mid = int(len(trade_ally_mid)/2)
simul_trade_ally_mid_reg = int(len(trade_ally_mid[trade_ally_mid['un_region_1'] == trade_ally_mid['un_region_2']])/2)
simul_trade_ally_mid_cont = int(len(trade_ally_mid[trade_ally_mid['un_continent_1'] == trade_ally_mid['un_continent_2']])/2)
simul_trade_ally_mid_co = int(len(trade_ally_mid['c_code_1'].unique()))

In [57]:
print('percent of trades between parties in alliances:', round(((simul_trade_ally/total_trades) * 100), 2), '%')
print('of trades between parties also in alliance, percent in same region:', round(((simul_trade_ally_reg/simul_trade_ally) * 100), 2), '%')
print('of trades between parties also in alliance, percent on same continent:', round(((simul_trade_ally_cont/simul_trade_ally) * 100), 2), '%')
print('of countries engaged in trade, percent engaged in trade and alliance:', round(((simul_trade_ally_co/total_trading_co) * 100), 2), '%')
print('')
print('percent of trades between parties in alliances and conflict:', round(((simul_trade_ally_mid/total_trades) * 100), 2), '%')
print('of trades between parties also in alliance and conflict, percent in same region:', round(((simul_trade_ally_mid_reg/simul_trade_ally_mid) * 100), 2), '%')
print('of trades between parties also in alliance and conflict, percent on same continent:', round(((simul_trade_ally_mid_cont/simul_trade_ally_mid) * 100), 2), '%')
print('of countries engaged in trade, percent engaged in trade, alliances, and conflict:', round(((simul_trade_ally_mid_co/total_trading_co) * 100), 2), '%')
print('')
print('percent of alliances between trading parties:', round(((simul_trade_ally/total_alliances) * 100), 2), '%')
print('of alliances between parties also in trade, percent in same region:', round(((simul_trade_ally_mid_reg/simul_trade_ally) * 100), 2), '%')
print('of alliances between parties also in trade, percent on same continent:', round(((simul_trade_ally_mid_cont/simul_trade_ally) * 100), 2), '%')
print('of countries in alliances, percent engaged in alliances and trade:', round(((simul_trade_ally_mid_co/total_alliance_co) * 100), 2), '%')
print('')
print('percent of alliances between parties in trade and conflict:', round(((simul_trade_ally_mid/total_alliances) * 100), 2), '%')
print('of alliances between parties also in trade and conflict, percent in same region:', round(((simul_trade_ally_mid_reg/simul_trade_ally_mid) * 100), 2), '%')
print('of alliances between parties also in trade and conflict, percent on same continent:', round(((simul_trade_ally_mid_cont/simul_trade_ally_mid) * 100), 2), '%')
print('of countries engaged in alliances, percent engaged in alliances, trade and conflict:', round(((simul_trade_ally_mid_co/total_alliance_co) * 100), 2), '%')
print('')
print('percent of conflicts between countries in alliance:', round(((simul_ally_mid/total_conflicts) * 100), 2), '%')
print('of conflicts conflicts between parties also in alliance, percent in same region:', round(((simul_ally_mid_reg/simul_ally_mid) * 100), 2), '%')
print('of conflicts conflicts between parties also in alliance, percent on same continent:', round(((simul_ally_mid_cont/simul_ally_mid) * 100), 2), '%')
print('of countries engaged in conflict, percent engaged in conflict and alliances:', round(((simul_ally_mid_co/total_conflict_co) * 100), 2), '%')
print('')
print('percent of conflicts between trading countries:', round(((simul_trade_mid/total_conflicts) * 100), 2), '%')
print('of conflicts between parties also in trade, percent in same region:', round(((simul_trade_mid_reg/simul_trade_mid) * 100), 2), '%')
print('of conflicts between parties also in trade, percent on same continent:', round(((simul_trade_mid_cont/simul_trade_mid) * 100), 2), '%')
print('of countries engaged in conflict, percent engaged in conflict and trade:', round(((simul_trade_mid_co/total_conflict_co) * 100), 2), '%')
print('')
print('percent of conflicts between parties in trade and alliance:', round(((simul_trade_ally_mid/total_conflicts) * 100), 2), '%')
print('of conflicts between parties in also trade and alliance, percent in same region:', round(((simul_trade_ally_mid_reg/simul_trade_ally_mid) * 100), 2), '%')
print('of conflicts between parties in also trade and alliance, percent on same continent:', round(((simul_trade_ally_mid_cont/simul_trade_ally_mid) * 100), 2), '%')
print('of countries engaged in conflict, percent engaged in conflict, trade and alliance:', round(((simul_trade_ally_mid_co/total_conflict_co) * 100), 2), '%')

percent of trades between parties in alliances: 8.76 %
of trades between parties also in alliance, percent in same region: 38.91 %
of trades between parties also in alliance, percent on same continent: 78.32 %
of countries engaged in trade, percent engaged in trade and alliance: 81.09 %

percent of trades between parties in alliances and conflict: 0.15 %
of trades between parties also in alliance and conflict, percent in same region: 65.37 %
of trades between parties also in alliance and conflict, percent on same continent: 75.16 %
of countries engaged in trade, percent engaged in trade, alliances, and conflict: 65.67 %

percent of alliances between trading parties: 92.24 %
of alliances between parties also in trade, percent in same region: 1.15 %
of alliances between parties also in trade, percent on same continent: 1.32 %
of countries in alliances, percent engaged in alliances and trade: 80.49 %

percent of alliances between parties in trade and conflict: 1.62 %
of alliances between 

In [61]:
aggregations = {'year': 'count'}
trade_mid_grouped = trade_mid.groupby(['c_code_1', 'c_code_2', 'dispute_id', 'date_start', 'date_end']).agg(aggregations).reset_index()

In [123]:
trade_mid_combos = list(trade_mid_grouped[['c_code_1', 'c_code_2']].values)
num_trade_yr_before_mid = []
num_trade_5_yr_before_mid = []
num_trade_10_yr_before_mid = []
num_trade_yr_after_mid = []
num_trade_5_yr_after_mid = []
num_trade_10_yr_after_mid = []
for combo in trade_mid_combos:
    before_yr = trade_mid_grouped[(trade_mid_grouped['c_code_1'] == combo[0]) & (trade_mid_grouped['c_code_2'] == combo[1])]['date_start'].dt.year.min() - 1
    before_yr_5 = trade_mid_grouped[(trade_mid_grouped['c_code_1'] == combo[0]) & (trade_mid_grouped['c_code_2'] == combo[1])]['date_start'].dt.year.min() - 5
    before_yr_10 = trade_mid_grouped[(trade_mid_grouped['c_code_1'] == combo[0]) & (trade_mid_grouped['c_code_2'] == combo[1])]['date_start'].dt.year.min() - 10
    after_yr = trade_mid_grouped[(trade_mid_grouped['c_code_1'] == combo[0]) & (trade_mid_grouped['c_code_2'] == combo[1])]['date_end'].dt.year.max() + 1
    after_yr_5 = trade_mid_grouped[(trade_mid_grouped['c_code_1'] == combo[0]) & (trade_mid_grouped['c_code_2'] == combo[1])]['date_end'].dt.year.max() + 5
    after_yr_10 = trade_mid_grouped[(trade_mid_grouped['c_code_1'] == combo[0]) & (trade_mid_grouped['c_code_2'] == combo[1])]['date_end'].dt.year.max() + 10
    if len(trade_df[(trade_df['c_code_1'] == combo[0]) & (trade_df['c_code_2'] == combo[1]) & (trade_df['year'] == before_yr)]) > 0:
        num_trade_yr_before_mid.append(1)
    else:
        num_trade_yr_before_mid.append(0)
    if len(trade_df[(trade_df['c_code_1'] == combo[0]) & (trade_df['c_code_2'] == combo[1]) & (trade_df['year'] == before_yr_5)]) > 0:
        num_trade_5_yr_before_mid.append(1)
    else:
        num_trade_5_yr_before_mid.append(0)
    if len(trade_df[(trade_df['c_code_1'] == combo[0]) & (trade_df['c_code_2'] == combo[1]) & (trade_df['year'] == before_yr_10)]) > 0:
        num_trade_10_yr_before_mid.append(1)
    else:
        num_trade_10_yr_before_mid.append(0)

    if len(trade_df[(trade_df['c_code_1'] == combo[0]) & (trade_df['c_code_2'] == combo[1]) & (trade_df['year'] == after_yr)]) > 0:
        num_trade_yr_after_mid.append(1)
    else:
        num_trade_yr_after_mid.append(0)
    if len(trade_df[(trade_df['c_code_1'] == combo[0]) & (trade_df['c_code_2'] == combo[1]) & (trade_df['year'] == after_yr_5)]) > 0:
        num_trade_5_yr_after_mid.append(1)
    else:
        num_trade_5_yr_after_mid.append(0)
    if len(trade_df[(trade_df['c_code_1'] == combo[0]) & (trade_df['c_code_2'] == combo[1]) & (trade_df['year'] == after_yr_10)]) > 0:
        num_trade_10_yr_after_mid.append(1)
    else:
        num_trade_10_yr_after_mid.append(0)

In [130]:
print('of the conflicts between trading countries, percent who traded 1 year before conflict started', round(((pd.Series(num_trade_yr_before_mid).sum()/len(num_trade_yr_before_mid)) * 100), 2), '%')
print('of the conflicts between trading countries, percent who traded 5 years before conflict started', round(((pd.Series(num_trade_5_yr_before_mid).sum()/len(num_trade_5_yr_before_mid)) * 100), 2), '%')
print('of the conflicts between trading countries, percent who traded 10 years before conflict started', round(((pd.Series(num_trade_10_yr_before_mid).sum()/len(num_trade_10_yr_before_mid)) * 100), 2), '%')
print('of the conflicts between trading countries, percent who traded 1 year after conflict ended', round(((pd.Series(num_trade_yr_after_mid).sum()/len(num_trade_yr_after_mid)) * 100), 2), '%')
print('of the conflicts between trading countries, percent who traded 5 years after conflict ended', round(((pd.Series(num_trade_5_yr_after_mid).sum()/len(num_trade_5_yr_after_mid)) * 100), 2), '%')
print('of the conflicts between trading countries, percent who traded 10 years after conflict ended', round(((pd.Series(num_trade_10_yr_after_mid).sum()/len(num_trade_10_yr_after_mid)) * 100), 2), '%')

of the conflicts between trading countries, percent who traded 1 year before conflict started 85.18 %
of the conflicts between trading countries, percent who traded 5 years before conflict started 66.19 %
of the conflicts between trading countries, percent who traded 10 years before conflict started 55.97 %
of the conflicts between trading countries, percent who traded 1 year after conflict ended 80.08 %
of the conflicts between trading countries, percent who traded 5 years after conflict ended 66.47 %
of the conflicts between trading countries, percent who traded 10 years after conflict ended 38.04 %


Network Analysis with Netowrkx

In [ ]:
df = pd.merge(alliance_net_df, mid_net_df, how = 'inner', on = ['c_code_1', 'c_code_2', 'year'])

In [ ]:
df = deepcopy(pd.merge(df, trade_net_df, how = 'inner', on = ['c_code_1', 'c_code_2', 'year']))

In [ ]:
df['avg_money_flow_log'] = np.log((df['money_flow_1'] + df['money_flow_2'])/2)
df['export_import_ratio'] = (df['money_flow_1'] / df['money_flow_2'])/2

In [ ]:
df.rename({'state_name_1_x': 'state_name_1', 'state_name_2_x': 'state_name_2'}, axis = 1, inplace = True)
df.drop(['state_name_1_y', 'state_name_2_y'], axis = 1, inplace = True)

In [ ]:
len(df)

In [ ]:
df = deepcopy(df[df['un_region_1'] != df['un_region_2']])

In [ ]:
len(df)

In [ ]:
len(df['state_name_1'].unique())

In [ ]:
df.head()

In [ ]:
combo_df = deepcopy(df[['state_name_1', 'state_name_2', 'year', 'date_start', 'avg_money_flow_log', 'export_import_ratio', 'cumulative_duration']])

In [ ]:
aggregations = {
    'date_start': 'count',
    'avg_money_flow_log': 'mean',
    'export_import_ratio': 'mean',
    'cumulative_duration': 'sum'
    }

group_combo_df = combo_df.groupby(['year', 'state_name_1', 'state_name_2']).agg(aggregations).reset_index()

In [ ]:
group_combo_df.head()

In [ ]:
aggregations = {
    'date_start': 'sum',
    'year': 'count',
    'avg_money_flow_log': 'mean',
    'export_import_ratio': 'mean',
    'cumulative_duration': 'sum'
    }

group_combo_df = group_combo_df.groupby(['state_name_1', 'state_name_2']).agg(aggregations).reset_index()

In [ ]:
for i, combo in enumerate(group_combo_df['state_name_1']):
    group_combo_df.loc[i, 'treaties_per_year'] = group_combo_df['date_start'][i]/group_combo_df['year'][i]
group_combo_df.rename({'year': 'num_years'}, axis = 1, inplace = True)
group_combo_df.drop(['date_start'], axis = 1, inplace = True)

In [ ]:
group_combo_df.head()

In [ ]:
distinct_combo_df = deepcopy(df[['state_name_1', 'state_name_2']])

duplicate_list = ['state_name_1', 'state_name_2']

distinct_combo_df.drop_duplicates(subset = duplicate_list, keep = 'first', inplace = True)

In [ ]:
distinct_combo_df = distinct_combo_df.reset_index()
distinct_combo_df.drop(['index'], axis = 1, inplace = True)

In [ ]:
all_combos = []
for i, row in enumerate(distinct_combo_df['state_name_1']):
    all_combos.append((distinct_combo_df['state_name_1'][i], distinct_combo_df['state_name_2'][i]))

In [ ]:
all_combos = sorted(all_combos)

In [ ]:
len(all_combos)

In [ ]:
distinct_countries = []
for i in flatten(all_combos):
    distinct_countries.append(i)
distinct_countries = sorted(list(set(distinct_countries)))

In [ ]:
len(distinct_countries)

In [ ]:
np.array(distinct_countries)

In [ ]:
combo_deg_1 = list(filter(lambda country: 'Canada' in country, all_combos))

In [ ]:
len(combo_deg_1)

In [ ]:
node_deg_1 = []
for a, b in combo_deg_1:
    node_deg_1.append(a)
    node_deg_1.append(b)
node_deg_1 = sorted(list(set(node_deg_1)))

In [ ]:
len(node_deg_1)

In [ ]:
full_network = []
for a, b in all_combos:
    for country in node_deg_1:
        if country == a:
            full_network.append((a, b))
        elif country == b:
            full_network.append((a, b))
full_network = sorted(full_network)

In [ ]:
len(full_network)

In [ ]:
for i, combo in enumerate(group_combo_df['state_name_1']):
    for z, connection in enumerate(full_network):
        if full_network[z][0] == combo and full_network[z][1] == group_combo_df['state_name_2'][i]:
            group_combo_df.loc[i, 'full_network'] = 'Yes'
            
full_network_df = group_combo_df[group_combo_df['full_network'] == 'Yes']
group_combo_df.drop(['full_network'], axis = 1, inplace = True)
full_network_df = full_network_df.reset_index()
full_network_df.drop(['index', 'full_network'], axis = 1, inplace = True)

In [ ]:
from collections import Counter

In [ ]:
full_network_counter = Counter(full_network)

In [ ]:
all_nodes = []
for country_a, country_b in full_network:
    all_nodes.append(country_a)
    all_nodes.append(country_b)
all_nodes = sorted(list(set(all_nodes)))

In [ ]:
len(all_nodes)

In [ ]:
full_network_counter

In [ ]:
full_network_df[(full_network_df['state_name_1'] == 'Republic of Vietnam') | (full_network_df['state_name_2'] == 'Republic of Vietnam')]['avg_money_flow_log']

In [ ]:
full_network_df.head()

In [ ]:
full_network_df[(full_network_df['state_name_1'] == 'Gerogia') | (full_network_df['state_name_2'] == 'Gerogia')]

In [ ]:
graph_file = open("./d3/network.json", 'w+')

json_dic = {}
json_dic['nodes'] = {}
json_dic['links'] = {}
start_line = '{\n  "nodes": [\n'

graph_file.write(start_line)
for i, node in enumerate(all_nodes):
    if node == all_nodes[-1]:
        add_line = '    {"country": "' + node + '", "size": ' + str(full_network_df[(full_network_df['state_name_1'] == node) | (full_network_df['state_name_2'] == node)]['avg_money_flow_log'].mean()) + '}\n'
        graph_file.write(add_line)
    else:
        add_line = '    {"country": "' + node + '", "size": ' + str(full_network_df[(full_network_df['state_name_1'] == node) | (full_network_df['state_name_2'] == node)]['avg_money_flow_log'].mean()) + '},\n'
        graph_file.write(add_line)        
middle_line = start_line = '  ],\n  "links": [\n'
graph_file.write(start_line)
node_count = 0
for node_1, node_2 in full_network:
    node_count+=1
    bond = full_network_counter[(node_1, node_2)]
    if node_count == len(full_network):
        add_line = '    {"source": ' + str(all_nodes.index(node_1)) + ', "target": ' + str(all_nodes.index(node_2)) + ', "bond": ' + str(bond) + '}\n'
        graph_file.write(add_line)
    else:
        add_line = '    {"source": ' + str(all_nodes.index(node_1)) + ', "target": ' + str(all_nodes.index(node_2)) + ', "bond": ' + str(bond) + '},\n'
        graph_file.write(add_line)
end_line = '\n  ]\n}'
graph_file.write(end_line)

In [ ]:
country_graph = nx.Graph()

In [ ]:
for country_a, country_b in all_combos:
    country_a_label = country_a.replace(' ', '\n')
    country_b_label = country_b.replace(' ', '\n')
    treaties_per_year = 1000 * (int(group_combo_df[(group_combo_df['state_name_1'] == country_a) & (group_combo_df['state_name_2'] == country_b)]['treaties_per_year'].values[0]))
    country_graph.add_edge(country_a_label, country_b_label, weight = treaties_per_year)

In [ ]:
filterwarnings('ignore')

plt.style.use('seaborn')
plt.figure(figsize = (12, 6))

positions = nx.spring_layout(country_graph)
network = nx.draw(country_graph, pos = positions)
labels = nx.draw_networkx_labels(country_graph, pos = positions)
nx.write_gexf(country_graph, "test.gexf")

In [ ]:
nx.number_of_nodes(country_graph)

In [ ]:
print(np.asarray(nx.degree(country_graph)))

In [ ]:
# Degree histogram of the network
degree_hist = nx.degree_histogram(country_graph)
xticks = range(len(degree_hist))
plt.bar(xticks, degree_hist, align='center')
plt.xlabel('Degree')
plt.ylabel('Number of nodes with this degree')
plt.title('An initial look into the degree distribution');

In [ ]:
# Network density
nx.density(country_graph)

In [ ]:
# There is a single component in this graph (in others, there can be many components)
for i, component in enumerate(nx.connected_components(country_graph)):
    print("COMPONENT {:d}:".format(i), component)

In [ ]:
# What are the degrees of France's neighbors? 
for neighbor in nx.all_neighbors(country_graph, 'United\nKingdom'):
    print(neighbor)
    degree = nx.degree(country_graph, neighbor)
    print('- degree: {:d}'.format(degree))
    print('------------')

In [ ]:
# Let's focus on other centrality values of these nodes

# Closeness: Inverse of the average length of all shortest paths from this node to all other nodes
# (closer to 1 = more central)
nx.closeness_centrality(country_graph)

In [ ]:
# Betweenness: Consider the shortest paths between each pair of nodes.
# What fraction of all these paths are going through this node?
nx.betweenness_centrality(country_graph)

In [ ]:
# Eigencentrality: like Google's PageRank: a measure of information flow
nx.eigenvector_centrality(country_graph)

In [ ]:
# (in case there are multiple paths of this same shortest length between these two)
# for i, path in enumerate(nx.all_shortest_paths(country_graph, 'Jordan', 'Ecuador')):
#     print("Shortest Path {}:".format(i), path)

In [ ]:
nx.average_shortest_path_length(country_graph)

In [ ]:
# Find the Communities with python-louvain
modules = community.best_partition(country_graph)
modules

In [ ]:
# How many modules do we have?
module_ids = set(modules.values())
num_modules = len(module_ids)
num_modules

In [ ]:
# It would be nice to look up a module and
# see which nodes its members are
from collections import defaultdict
module_to_nodes = defaultdict(list)
for node, module_id in modules.items():
    module_to_nodes[module_id].append(node)

module_to_nodes

In [ ]:
# Let's visualize the modules

# Use the same layout as before
positions = nx.spring_layout(country_graph)

# But now, instead of drawing it all at once, draw
# nodes module by module, using a different color
# for each module
colors = ['m', 'c', 'y', 'g', 'r', 'b']
for i, module in enumerate(module_ids):
    nodes_in_this_module = module_to_nodes[module]
    module_color = colors[i % len(colors)]
    nx.draw_networkx_nodes(country_graph,
                           positions,
                           nodes_in_this_module,
                           node_color = module_color)
    
# draw the edges and labels as well and voila
nx.draw_networkx_edges(country_graph, positions, alpha=0.5)
labels = nx.draw_networkx_labels(country_graph, pos = positions)

In [ ]:
nx.write_gexf(country_graph, "test.gexf")